# 📘 Desafio 06 - Empréstimos por Curso (2015-2020)

## 🎯 Objetivo

Neste desafio, vamos identificar quantos empréstimos foram realizados entre os anos de 2015 e 2020 pelos cursos de graduação avaliados pelo MEC. Para isso, precisaremos combinar informações de diferentes fontes (Excel e JSON) com os dados de empréstimos consolidados anteriormente.

Cursos analisados:

- Biblioteconomia
- Ciências Sociais
- Comunicação Social
- Direito
- Filosofia
- Pedagogia

In [2]:
# 📚 Importando bibliotecas essenciais

import pandas as pd
import numpy as np

## 📥 Importação dos Dados de Usuários

Vamos importar os dados dos alunos vindos de duas fontes distintas:

- Um arquivo `.xlsx` com parte das matrículas
- Um arquivo `.json` com registros adicionais

Esses dados serão combinados em um único DataFrame para posterior junção com os empréstimos.

In [4]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from pathlib import Path

# Criando a estrutura de pastas local
pasta_dados = Path('dados')
pasta_dados.mkdir(parents=True, exist_ok=True)

In [11]:
import requests

# URLs dos arquivos
url_excel = "https://raw.githubusercontent.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas/main/Dia_6-Novos_dados_novas_analises/Datasets/matricula_alunos.xlsx"
url_json = "https://raw.githubusercontent.com/FranciscoFoz/7_Days_of_Code_Alura-Python-Pandas/main/Dia_6-Novos_dados_novas_analises/Datasets/cadastro_alunos.json"

# Download Excel
r_excel = requests.get(url_excel)
with open("dados/matricula_alunos.xlsx", "wb") as f:
    f.write(r_excel.content)

# Download JSON
r_json = requests.get(url_json)
with open("dados/cadastro_alunos.json", "wb") as f:
    f.write(r_json.content)

In [12]:
# Importando os dados Excel
matricula_alunos = pd.read_excel('dados/matricula_alunos.xlsx')

# Importando os dados JASON
cadastro_alunos = pd.read_json('dados/cadastro_alunos.json')

# Importando o dataset completo gerado no desafio04
df = pd.read_csv('../desafio04/dataset_completo.csv')

# Visualizando as primeiras linhas
display(df.head(3), matricula_alunos.head(3), cadastro_alunos.head(3))

C:\Users\Jhonny\AppData\Local\Temp\ipykernel_16952\3463228346.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../desafio04/dataset_completo.csv')


,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,classe_cdu
0,709684,L095049,NaN,2010-01-04 07:44:10.721,2010-01-05 16:26:12.662,2.008023e+09,ALUNO DE GRADUAÇÃO,13259.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630,Ciências aplicadas
1,709685,L167050,NaN,2010-01-04 07:44:10.750,2010-01-12 07:34:13.934,2.008023e+09,ALUNO DE GRADUAÇÃO,70865.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647,Ciências aplicadas
2,709686,2006017618,2010-01-26 08:07:01.738,2010-01-04 08:08:44.081,2010-02-25 07:36:25.800,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO,195347.0,Acervo Circulante,Biblioteca Setorial Prof. Rodolfo Helinski - E...,REGULAR,640,Ciências aplicadas


,Matricula de alunos até 2010,Unnamed: 1,Unnamed: 2
0,Número da matrícula,Tipo de vinculo,Curso
1,190001688.0,ALUNO DE GRADUAÇÃO,MATEMÁTICA
2,199014489.0,ALUNO DE GRADUAÇÃO,CIÊNCIAS BIOLÓGICAS


,tipo_curso,registros
0,GRADUACAO,"[{""matricula_ou_siape"": ""2009029003.0"", ""tipo_..."
1,POSGRADUACAO,"[{""matricula_ou_siape"": ""2008112114.0"", ""tipo_..."


## 🧼 Pré-processamento dos Dados de Matrícula

A fim de enriquecer a análise com a variável *curso dos alunos*, recebemos dois conjuntos de dados complementares:

1. **`matricula_alunos.xlsx`** – Planilha contendo dados de alunos da graduação até o ano de 2010.
2. **`cadastro_alunos.json`** – Arquivo com informações de alunos de graduação e pós-graduação matriculados após 2010, estruturado por meio de registros em listas aninhadas.

### Estrutura dos dados

**Excel (`.xlsx`)**
- Possui duas linhas de cabeçalho desnecessárias: a primeira é um título e a segunda contém os nomes das colunas.
- As colunas válidas são:
  - `Número da matrícula`
  - `Tipo de vinculo`
  - `Curso`
- A matrícula aparece como número com ponto flutuante (ex: `190001688.0`), o que exige padronização para `string`.

**JSON (`.json`)**
- Apresenta duas linhas: uma para *graduação* e outra para *pós-graduação*.
- Cada linha contém, na coluna `registros`, uma lista de dicionários com as seguintes chaves:
  - `matricula_ou_siape`
  - `tipo_vinculo_usuario`
  - `curso`

### Objetivo do tratamento

Nosso objetivo é obter um **único DataFrame unificado** contendo:
- `matricula_ou_siape`
- `tipo_vinculo_usuario`
- `curso`

Esse novo DataFrame servirá como base para cruzar com o dataset de empréstimos e, assim, realizar a análise por curso solicitada neste desafio.

Vamos começar o tratamento a partir do Excel.

In [15]:
# Tratando os dados Excel

# Pulando a primeira linha, para obter o cabeçalho corretamente
df_excel = pd.read_excel("dados/matricula_alunos.xlsx", skiprows=1)

# Renomeando as colunas de forma limpa
df_excel.columns = ['matricula_ou_siape', 'tipo_vinculo_usuario', 'curso']

# Transformando a matrícula em string
df_excel['matricula_ou_siape'] = df_excel['matricula_ou_siape'].astype(str)

df_excel.head(3)

,matricula_ou_siape,tipo_vinculo_usuario,curso
0,190001688.0,ALUNO DE GRADUAÇÃO,MATEMÁTICA
1,199014489.0,ALUNO DE GRADUAÇÃO,CIÊNCIAS BIOLÓGICAS
2,199735674.0,ALUNO DE GRADUAÇÃO,MATEMÁTICA


In [ ]:
import json

# Caminho local do JSON
caminho_json = 'dados/cadastro_alunos.json'

# Abrindo e carregando com json
with open(caminho_json, 'r', encoding='utf-8') as f:
    dados_json = json.load(f)

# Verificando a estrutura
for i, item in enumerate(dados_json):
    print(f"\n🔎 ITEM {i}")
    print("tipo_curso:", item.get('tipo_curso'))
    print("registros (tipo):", type(item.get('registros')))
    print("registros (exemplo):", item.get('registros')[:2])  # mostra só os 2 primeiros


🔎 ITEM 0
tipo_curso: GRADUACAO
registros (tipo): <class 'str'>
registros (exemplo): [{

🔎 ITEM 1
tipo_curso: POSGRADUACAO
registros (tipo): <class 'str'>
registros (exemplo): [{


In [27]:
import json

with open("dados/cadastro_alunos.json", "r", encoding="utf-8") as file:
    dados_json = json.load(file)

lista_dataframes = []

for item in dados_json:
    registros_str = item.get('registros')

    try:
        # Decodifica a string JSON interna
        registros = json.loads(registros_str)

        # Agora sim é uma lista de dicionários
        if isinstance(registros, list) and all(isinstance(r, dict) for r in registros):
            temp_df = pd.DataFrame(registros)
            lista_dataframes.append(temp_df)
        else:
            print("❌ Estrutura inválida após json.loads")

    except Exception as e:
        print(f"⚠️ Erro ao decodificar registros: {e}")

# Concatena os DataFrames válidos
df_json = pd.concat(lista_dataframes, ignore_index=True)

# Garantindo consistência no tipo de dado
df_json['matricula_ou_siape'] = df_json['matricula_ou_siape'].astype(str)

display(df_json.head())

,matricula_ou_siape,tipo_vinculo_usuario,curso
0,2009029003.0,ALUNO DE GRADUAÇÃO,FILOSOFIA
1,200617346.0,ALUNO DE GRADUAÇÃO,DIREITO
2,200746502.0,ALUNO DE GRADUAÇÃO,ENGENHARIA DE PETRÓLEO
3,200504968.0,ALUNO DE GRADUAÇÃO,ADMINISTRAÇÃO
4,200615572.0,ALUNO DE GRADUAÇÃO,CIÊNCIA DA COMPUTAÇÃO


In [43]:
# Concatenando as bases Excel e JSON
df_alunos = pd.concat([df_excel, df_json], ignore_index=True)
df_alunos.to_csv('dados/df_alunos.csv')

# Verificando amostras
print(df_alunos.shape)
df_alunos.sample(5)

(89968, 3)


,matricula_ou_siape,tipo_vinculo_usuario,curso
53440,20160124247.0,ALUNO DE GRADUAÇÃO,ENGENHARIA CIVIL
19172,2011010591.0,ALUNO DE GRADUAÇÃO,COMUNICAÇÃO SOCIAL
60103,20150125465.0,ALUNO DE GRADUAÇÃO,ENGENHARIA FLORESTAL
66341,2015019094.0,ALUNO DE GRADUAÇÃO,CIÊNCIAS ATUARIAIS
79022,2014107601.0,ALUNO DE PÓS-GRADUAÇÃO,ARQUITETURA E URBANISMO


### 🔗 Unificação dos Dados dos Alunos

Após o tratamento dos dados extraídos do Excel e JSON, unificamos ambos em um único DataFrame, `df_alunos`. Esse conjunto representa a base de usuários com seus vínculos institucionais e cursos de origem, necessária para cruzarmos com o histórico de empréstimos entre 2015 e 2020.

### 🎯 Filtrando Alunos dos Cursos Avaliados (Graduação)

Como parte da avaliação dos cursos de graduação pela instituição e pelo MEC, precisamos focar a análise nos seguintes cursos, que passarão por inspeção quanto ao uso da biblioteca:

- Biblioteconomia
- Ciências Sociais
- Comunicação Social
- Direito
- Filosofia
- Pedagogia

Nosso objetivo é identificar os alunos de graduação que estão matriculados nesses cursos e, em seguida, cruzar suas matrículas com os registros de empréstimos realizados entre 2015 e 2020.

Para isso, faremos dois filtros no nosso DataFrame consolidado df_alunos:

Filtrar apenas os registros cujo vínculo seja "ALUNO DE GRADUAÇÃO".

Selecionar apenas os alunos matriculados nos cursos citados acima.

In [32]:
# Criando uma lista dos cursos que serão avaliados
cursos_interesse = [
    'BIBLIOTECONOMIA',
    'CIÊNCIAS SOCIAIS',
    'COMUNICAÇÃO SOCIAL',
    'DIREITO',
    'FILOSOFIA',
    'PEDAGOGIA'
]

# Fazendo o filtro do `df_alunos`, tanto pelo curso, quanto pelo tipo de vínculo do aluno
df_graduacao = df_alunos[
    (df_alunos['tipo_vinculo_usuario'] == 'ALUNO DE GRADUAÇÃO') & # Toma somente os alunos da graduação
    (df_alunos['curso'].str.upper().isin(cursos_interesse)) # Verifica se o curso pertence á lista de cursos criada
]

display(df_graduacao.head(5), df_graduacao['curso'].value_counts())

,matricula_ou_siape,tipo_vinculo_usuario,curso
8,200031011.0,ALUNO DE GRADUAÇÃO,DIREITO
9,200045080.0,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
15,200115053.0,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS
18,200123382.0,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
19,200123730.0,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA


curso
COMUNICAÇÃO SOCIAL    3211
BIBLIOTECONOMIA       3186
CIÊNCIAS SOCIAIS      3180
DIREITO               3122
FILOSOFIA             2874
PEDAGOGIA             2847
Name: count, dtype: int64

## 🎯 Cruzamento entre Empréstimos e Alunos Avaliados

Agora que temos uma base consolidada com os alunos de graduação dos **cursos avaliados** (`Biblioteconomia`, `Ciências Sociais`, `Comunicação Social`, `Direito`, `Filosofia` e `Pedagogia`), precisamos **relacioná-los aos dados de empréstimos** para calcular a quantidade de empréstimos realizados por curso e por ano.

### 📌 Objetivo
Nosso objetivo é gerar uma **tabela resumo** contendo:
- **Índice:** Cursos avaliados
- **Colunas:** Anos (de 2015 a 2020)
- **Valores:** Quantidade de empréstimos por curso e por ano
- **Extras:** Linha de totais por ano e coluna de totais por curso

### 🧪 Etapa atual: Verificação de compatibilidade

Antes de cruzar os dados, precisamos garantir que os dois DataFrames envolvidos (`df` com os empréstimos e `df_alunos_filtrados` com os alunos dos cursos avaliados) tenham a mesma estrutura na coluna que será usada como chave de junção: `matricula_ou_siape`.

Para isso, faremos:
- 🔍 **Verificação do tipo da coluna `matricula_ou_siape`** em ambos os DataFrames.
- 🧼 Caso necessário, converteremos ambos para o mesmo tipo (preferencialmente `str`).
- ✅ Teste inicial para checar se há correspondência entre as matrículas dos alunos e as matrículas presentes na base de empréstimos.

### ✅ Próximo passo
Após essa validação, poderemos aplicar um filtro no DataFrame de empréstimos, mantendo **apenas os registros realizados pelos alunos desses cursos**, no intervalo de **2015 a 2020**, e prosseguir com a contagem dos empréstimos por ano.

In [34]:
# Verificando a compatibilidade entre os DataFrames
df['matricula_ou_siape'].dtype, df_graduacao['matricula_ou_siape'].dtype

(dtype('float64'), dtype('O'))

Obtivemos tipos diferentes para a coluna `matricula_ou_siape` para os DataFrames, o que indica a necessidade de conversão na base `df`, para o tipo `str`.

In [35]:
# Convertendo a coluna `matricula_ou_siape` em `str`
df['matricula_ou_siape'] = df['matricula_ou_siape'].astype(str)
df['matricula_ou_siape'].dtype

dtype('O')

## 🧹 Filtrando os Empréstimos por Cursos e Período

Com a base de alunos devidamente filtrada, agora é o momento de aplicar o mesmo filtro sobre os **dados de empréstimos**.

### 🎯 Objetivo
Manter no DataFrame de empréstimos (`df`) **apenas os registros que atendem simultaneamente** aos seguintes critérios:
1. **Pertencem a alunos dos cursos avaliados** de graduação.
2. **Foram realizados entre os anos de 2015 e 2020**, intervalo de interesse definido pelo desafio.

### 🔑 Lógica de Filtragem
- Usaremos a coluna `matricula_ou_siape` como chave para cruzar os dados.
- Aplicaremos um filtro que mantém apenas as linhas do DataFrame de empréstimos em que:
  - A matrícula do empréstimo está presente na base `df_graduacao`.
  - A data do empréstimo está entre `2015-01-01` e `2020-12-31`.

### 🧪 Antes de aplicar:
- Certificaremos que as colunas `matricula_ou_siape` e `data_emprestimo` estejam no formato apropriado (`str` e `datetime`, respectivamente).
- Em seguida, aplicaremos os filtros combinados e criaremos um novo DataFrame contendo apenas os registros relevantes para análise.

In [37]:
# Mesclando os DataFrames
df_emprestimos_filtrado = pd.merge(
    df,  # dados de empréstimos
    df_graduacao,  # dados de alunos dos cursos de interesse
    on='matricula_ou_siape', # vínculo entre os DataFrames
    how='inner'  # inner porque queremos apenas as matrículas que têm correspondência
)
df_emprestimos_filtrado.head(5)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario_x,id_exemplar,colecao,biblioteca,status_material,localizacao,classe_cdu,tipo_vinculo_usuario_y,curso
0,709698,2009047725,2010-01-18 14:44:41.163,2010-01-04 09:21:19.099,2010-02-02 12:02:38.444,2009046838.0,ALUNO DE GRADUAÇÃO,281004.0,Acervo Circulante,Biblioteca Setorial do Centro Ciências da Saúd...,REGULAR,651,Ciências aplicadas,ALUNO DE GRADUAÇÃO,DIREITO
1,709699,2009051872,2010-01-18 14:44:41.144,2010-01-04 09:21:19.126,2010-02-02 12:03:35.649,2009046838.0,ALUNO DE GRADUAÇÃO,281880.0,Acervo Circulante,Biblioteca Setorial do Centro Ciências da Saúd...,REGULAR,647,Ciências aplicadas,ALUNO DE GRADUAÇÃO,DIREITO
2,709724,2009022118,NaN,2010-01-04 10:44:44.282,2010-01-28 10:53:00.346,2009064639.0,ALUNO DE GRADUAÇÃO,259469.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,602,Ciências aplicadas,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
3,709725,2009022349,NaN,2010-01-04 10:44:44.311,2010-01-28 10:53:21.511,2009064639.0,ALUNO DE GRADUAÇÃO,270247.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,625,Ciências aplicadas,ALUNO DE GRADUAÇÃO,BIBLIOTECONOMIA
4,709730,2006040088,2010-01-19 09:16:57.265,2010-01-04 10:57:59.104,2010-02-12 11:30:47.029,200633139.0,ALUNO DE GRADUAÇÃO,159996.0,Acervo Circulante,Biblioteca Setorial do Centro Ciências da Saúd...,REGULAR,628,Ciências aplicadas,ALUNO DE GRADUAÇÃO,COMUNICAÇÃO SOCIAL


Agora, precisamos filtrar apenas o período entre 2015 à 2020. Para isso, primeiro garantimos que as colunas contendo data estão no formato `datetime`.

In [38]:
# Garante que a coluna está no tipo datetime
df_emprestimos_filtrado['data_emprestimo'] = pd.to_datetime(df_emprestimos_filtrado['data_emprestimo'], errors='coerce')

In [40]:
# Define intervalo de interesse
inicio = '2015-01-01'
fim = '2020-12-31'

# Aplica o filtro
df_emprestimos_filtrado = df_emprestimos_filtrado[
    (df_emprestimos_filtrado['data_emprestimo'] >= inicio) &
    (df_emprestimos_filtrado['data_emprestimo'] <= fim)
]

df_emprestimos_filtrado.head(5)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario_x,id_exemplar,colecao,biblioteca,status_material,localizacao,classe_cdu,tipo_vinculo_usuario_y,curso
205862,1895308,2014006710,NaN,2015-01-02 10:19:38.087,NaN,2014021454.0,ALUNO DE GRADUAÇÃO,1109169.0,Acervo Circulante,Biblioteca Setorial Prof. Ronaldo Xavier de Ar...,REGULAR,664,Ciências aplicadas,ALUNO DE GRADUAÇÃO,FILOSOFIA
205863,1895310,2014055802,2015-01-17 12:31:05.134,2015-01-02 10:25:40.883,NaN,2013018187.0,ALUNO DE GRADUAÇÃO,1164232.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,153,Filosofia e psicologia,ALUNO DE GRADUAÇÃO,DIREITO
205864,1895326,2008009575,2015-01-17 09:29:54.302,2015-01-02 12:26:43.571,NaN,2014029499.0,ALUNO DE GRADUAÇÃO,240242.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,612,Ciências aplicadas,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS
205865,1895327,2010072725,2015-01-17 09:29:53.780,2015-01-02 12:26:43.840,NaN,2014029499.0,ALUNO DE GRADUAÇÃO,824595.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,621,Ciências aplicadas,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS
205866,1895328,2012058303,2015-01-17 09:29:53.313,2015-01-02 12:26:44.750,NaN,2014029499.0,ALUNO DE GRADUAÇÃO,1024730.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,391,Ciências sociais,ALUNO DE GRADUAÇÃO,CIÊNCIAS SOCIAIS


### 📊 Geração da Tabela: Empréstimos por Curso e por Ano (2015–2020)

Agora que temos um DataFrame consolidado com:

- alunos da graduação nos cursos de interesse;
- registros de empréstimos entre 2015 e 2020;

...nosso próximo objetivo é construir uma tabela dinâmica com a quantidade de empréstimos realizados por curso em cada ano.

#### 🧩 Estrutura desejada:

- Índice: curso
- Colunas: anos (de 2015 a 2020)
- Valores: contagem de empréstimos (id_emprestimo)

Além disso, iremos adicionar:

- uma coluna de total por curso (linha horizontal);
- uma linha de total por ano (coluna vertical).

In [42]:
# 1. Criando uma coluna com o ano do empréstimo
df_emprestimos_filtrado['ano'] = df_emprestimos_filtrado['data_emprestimo'].dt.year

# 2. Criando a tabela dinâmica (pivot_table)
tabela_emprestimos = pd.pivot_table(
    df_emprestimos_filtrado,
    index='curso',
    columns='ano',
    values='id_emprestimo',
    aggfunc='count',
    fill_value=0  # substitui valores nulos por 0
)

# 3. Adicionando a coluna "Total" (por curso)
tabela_emprestimos['Total'] = tabela_emprestimos.sum(axis=1)

# 4. Adicionando a linha "Total" (por ano)
tabela_emprestimos.loc['Total'] = tabela_emprestimos.sum()

# Exibindo a tabela final
tabela_emprestimos


ano,2015,2016,2017,2018,2019,2020,Total
curso,,,,,,,
BIBLIOTECONOMIA,7989,7319,7661,6661,6264,945,36839
CIÊNCIAS SOCIAIS,7783,7294,7269,6472,6061,860,35739
COMUNICAÇÃO SOCIAL,7548,8488,8761,7336,6210,836,39179
DIREITO,7866,7599,8171,7249,6119,937,37941
FILOSOFIA,7362,7951,7776,6623,5872,865,36449
PEDAGOGIA,7579,7545,7888,6809,6186,916,36923
Total,46127,46196,47526,41150,36712,5359,223070


### 📊 Análise Final – Empréstimos por Curso (2015 a 2020)

Com a consolidação dos dados de usuários e empréstimos, foi possível gerar uma tabela cruzada com os **cursos de graduação de interesse** e a **quantidade de empréstimos realizados entre os anos de 2015 e 2020**.

| Curso               | 2015 | 2016 | 2017 | 2018 | 2019 | 2020 | Total |
|---------------------|------|------|------|------|------|------|--------|
| Biblioteconomia     | 7989 | 7319 | 7661 | 6661 | 6264 |  945 |  36839 |
| Ciências Sociais    | 7783 | 7294 | 7269 | 6472 | 6061 |  860 |  35739 |
| Comunicação Social  | 7548 | 8488 | 8761 | 7336 | 6210 |  836 |  39179 |
| Direito             | 7866 | 7599 | 8171 | 7249 | 6119 |  937 |  37941 |
| Filosofia           | 7362 | 7951 | 7776 | 6623 | 5872 |  865 |  36449 |
| Pedagogia           | 7579 | 7545 | 7888 | 6809 | 6186 |  916 |  36923 |
| **Total**           |46127 |46196 |47526 |41150 |36712 | 5359 | 223070 |

#### 📌 Principais Insights

- **Comunicação Social** lidera o ranking em número total de empréstimos no período, com **39.179** registros, seguido de **Direito** e **Pedagogia**.
- A partir de **2018**, nota-se um **declínio progressivo** no volume de empréstimos em todos os cursos, tendência que se acentua drasticamente em **2020**, possivelmente devido ao início da pandemia de COVID-19.
- O comportamento semelhante entre os cursos ao longo dos anos sugere que fatores externos impactaram o uso da biblioteca de forma generalizada, não havendo grandes disparidades entre as áreas.
- Mesmo em 2020, houve algum nível de uso, o que pode indicar a presença de serviços ainda operando (em menor escala) ou atrasos na devolução registrados como movimentação.

#### 📈 Sugestão

Uma análise complementar pode incluir:
- O cruzamento com os meses de maior movimentação por curso.
- A comparação com o tamanho do corpo discente de cada curso, para medir **proporcionalmente** o uso da biblioteca.
- A verificação da existência de campanhas institucionais de incentivo à leitura ou exigência de leituras específicas em disciplinas-chave ao longo dos anos.

---